# Fandango Seating Scraper
---

## Resources
* Web Scraping Tutorial: https://youtu.be/52wxGESwQSA
    * Associated GitHub repo: https://github.com/paulproteus/python-scraping-code-samples
* MechanicalSoup Tutorial: http://mechanicalsoup.readthedocs.io/en/stable/tutorial.html
* BeautifulSoup4 Documentation: https://www.crummy.com/software/BeautifulSoup/bs4/doc/
* Some Selenium stuff: http://stanford.edu/~mgorkove/cgi-bin/rpython_tutorials/Scraping_a_Webpage_Rendered_by_Javascript_Using_Python.php
* Add chromedriver to path: https://youtu.be/dz59GsdvUF8
* Fandango API: https://developer.fandango.com/docs/read/Fandango

In [1]:
import re
import os
import json
from time import time
from time import sleep
from datetime import datetime
import requests
import threading
import queue

from selenium import webdriver
from bs4 import BeautifulSoup

import mechanicalsoup as msoup
import lxml

## Selenium Solution (slow, but effective)

In [2]:
def params_to_url_string(params, url_base):
    '''
    Generate a URL string using a base url and a dictionary of URL
    GET request query parameters.
    
    ARGUMENTS:
    params -- URL parameters for a GET query
    url_base -- URL to query
    
    RETURN:
    url -- constructed URL from base URL and query parameters
    '''
    
    params_string = "&".join(["%s=%s" % (key, params[key]) for key in params])
    url = "?".join([url_base, params_string])
    
    return url

def collect_showing_type_id(showing_type_id, description):
    '''
    Add a showing type {ID: DESCRIPTION} pair to a centralized database
    of Fandango showing type IDs. (e.g. {"461654871": "RealD 3D, Reserved 
    seating, No passes, Closed caption"})
    
    ARGUMENTS:
    showing_type_id -- identification string (usually numeric)
    description -- string describing the type of showing to which the
        showing_type_id string refers.
    
    RETURNS:
    showing_type_ids -- dictionary of all collected showing-type IDs
    '''
    
    metadata_dir = "../data/meta"
    file_name = "showing_type_ids.json"
    file_path = os.path.join(metadata_dir, file_name)
    
    if not os.path.exists(metadata_dir):
        os.makedirs(metadata_dir)
        
    showing_type_ids = {}
    
    if os.path.exists(file_path):
        with open(file_path, "r") as fp:
            showing_type_ids = json.load(fp)
        
    if showing_type_id not in showing_type_ids:
        showing_type_ids[showing_type_id] = description
        with open(file_path, "w") as fp:
            json.dump(showing_type_ids, fp)
    
    return showing_type_ids

def get_movie_info(url_base, params):
    '''
    Get movie times for a selected theater and movie.
    
    ARGUMENTS:
    params -- URL parameters for a GET query
    url_base -- URL to query
    
    RETURNS:
    movie_info -- dictionary with the following keys:
        "dropdowndates" -- raw dictionary from Fandango source JavaScript,
        "showing_type_ids" -- dictionary of all collected showing-type IDs,
        "movie_title" -- string, title of movie as specified by Fandango,
        "theater_name" -- string, name of theater as specified by Fandango,
        "error" -- string, error encountered when trying to fetch movie info (default: None)
    '''
    
    movie_info = {
        "dropdowndates": None,
        "showing_type_ids": None,
        "movie_title": None,
        "theater_name": None,
        "error": None
    }
    
    response = requests.get(url = url_base, params = params)
    content = response.content.decode("utf-8")
    match = re.search(r"var dropdowndates = ({[\s\S]+]})\/\/\]\]", content)
    dropdowndates = json.loads(match.group(1)) if match else None
    
    if not dropdowndates:
        try:
            soup = BeautifulSoup(content, "html.parser")
            movie_info["error"] = soup.find("div", class_="errorHeaderMessage").text.strip()
            return movie_info
        except:
            movie_info["error"] = "UNKNOWN ERROR"
            return movie_info
    
    for date in dropdowndates:
        for item in dropdowndates[date]:
            description = item["Key"]
            showing_type_id = item["Times"][0].split("_")[1]
            showing_type_ids = collect_showing_type_id(showing_type_id, description)
    
    match = re.search(r'<h3 id="movieTitle">([\s\S]+?)</h3>', content)
    movie_title = match.group(1) if match else "movie_title_unknown"
    movie_title = re.sub(r"[*.\"\/\\\[\]:;|=,\!]", "_", movie_title).strip(" _")
    
    match = re.search(r'<h2 id="theaterName">([\s\S]+?)</h2>', content)
    theater_name = match.group(1) if match else "theater_name_unknown"
    theater_name = re.sub(r"[*.\"\/\\\[\]:;|=,\!]", "_", theater_name).strip(" _")
    
    movie_info = {
        "dropdowndates": dropdowndates,
        "showing_type_ids": showing_type_ids,
        "movie_title": movie_title,
        "theater_name": theater_name,
        "error": None
    }
    
    return movie_info

def click_through(url, movie_date, movie_time, op_sys="win", browser=None):
    '''
    Click through the seating reservation process using Selenium to find and download
    seating reservation status for all seats at a particular movie showing.
    
    ARGUMENTS: 
    url -- URL to datetime selection page for a particular movie and theater
    movie_date -- string, movie date in the format specified by Fandango
    movie_time -- string, movie time in the format specified by Fandango
    op_sys -- "win" or "linux", operating system of machine running this code (default: "win") 
    browser -- current Selenium webdriver object (default: None)
    
    RETURNS:
    browser -- current Selenium webdriver object
    page_info -- dictionary with the following keys:
        "source" -- string, source code fetched from clicked-to webpage,
        "error" -- string, error encountered when trying to click through webpages (default: None)
    '''
    
    page_info = {
        "source": None,
        "error": None
    }
    
    chromedriver_path = {
        "win": "../webdriver/chromedriver.exe",
        "linux": "../webdriver/chromedriver_linux64"
    }
    
    if browser:
        
        try: 
            # Click "Select new showtime"
            browser.find_element_by_xpath("//*[@id='movieTicketSummary']/div[3]/ul[2]/li[2]/p/a").click()
        except:
            browser.close()
            browser = webdriver.Chrome(chromedriver_path[op_sys])
            browser.get(url = url)
    
    else:
        browser = webdriver.Chrome(chromedriver_path[op_sys])
        browser.get(url = url)
    
    browser.find_element_by_xpath("//*[@id='ShowDateDropDownList']/option[@value='%s']" % movie_date).click()
    browser.find_element_by_xpath("//*[@id='ShowTimeDropDownList']/optgroup/option[@value='%s']" % movie_time).click()
    
    try:
        browser.find_element_by_xpath("//select[@name='AreaRepeater$ctl00$TicketRepeater$ctl00$quantityddl']/option[@value='1']").click()
    
    except:
        page_info["error"] = browser.find_element_by_class_name("errorHeaderMessage").text + "\n"
        return browser, page_info
    
    browser.find_element_by_id("NewCustomerCheckoutButton").click()
    
    try:
        browser.find_element_by_xpath("//*[@id='completePurchaseButton']")
        page_info["error"] = "This showing does not support reserved seating.\n"
        return browser, page_info
    
    except:
        page_info["source"] = browser.page_source
    
    return browser, page_info

def find_and_parse_seat_tags(source):
    '''
    If they are present, find and parse the seat tags in the seat selection page's
    source code.
    
    ARGUMENT:
    source -- string, source code fetched from final clicked-to webpage
    
    RETURNS:
    seat_info -- dictionary with the following keys:
        "seats" -- lists of seats in auditorium (i.e. list of string concatenations 
            of row letter and column number for every seat in auditorium), arranged
            in the order in which the seat tags appear in the Fandango source code
        "auditorium" -- string, auditorium name as specified by Fandango,
        "error" -- string, error encountered when trying to parse seating data
    '''
    
    seats = None
    error = None
    
    soup = BeautifulSoup(source, "html.parser")
    auditorium = soup.find("h2", id="auditoriumInfo").text.strip()
    seat_tags = soup.findAll(name = "div", attrs = {"class": re.compile(".*Seat"), "id": re.compile(".*")})
    
    if seat_tags:
        
        a = "availableSeat"
        u = "unavailableSeat"
        
        seats_raw = [[seat["id"], *seat["class"]] for seat in seat_tags]
        seats = [[x[0], x[1], "A" if x[-1] == a else "U" if x[-1] == u else "R"] for x in seats_raw]
        
    else:
        
        error = soup.find(name = "div", attrs = {"class": "errorHeaderMessage"}).get_text(strip = True)
        
        if not error:
            error = "Seating chart not found."
    
    seat_info = {
        "seats": seats,
        "auditorium": auditorium,
        "error": error
    }
    
    return seat_info

def get_paths(params, movie_info, movie_datetime):
    '''
    Generate file paths given movie paramater information.
    
    ARGUMENTS:
    params -- dictionary containing key "tid" (a 5-letter theater ID)
    movie_info -- dictionary containing keys ["movie_title", "theater_name"]
    movie_datetime -- full datetime string, including Fandango showing-type-id
    
    RETURNS:
    paths -- dictionary containing the following keys:
        "data_dir" -- path to directory containing movie data file,
        "data_file" -- path to movie data file,
        "log_file" -- path to movie data log file
    '''
    
    data_dir = "../data/movies/%s/%s - %s" % (movie_info["movie_title"], params["tid"], movie_info["theater_name"])
    data_file = os.path.join(data_dir, "%s.txt" % movie_datetime)
    log_file = os.path.join(data_dir, "%s.log" % movie_datetime)
    
    paths = {
        "data_dir": data_dir,
        "data_file": data_file,
        "log_file": log_file
    }
    
    return paths

def write_log(event, log_time, log_text_dict, log_file):
    '''
    Write event data to log file. Create log file if one doesn't already exist.
    
    ARGUMENTS:
    event -- string indicating event to be logged
    log_time -- string, time of log entry
    log_text_dict -- dictionary containing default log phrases for each possible event
    log_file -- path to log file
    
    RETURNS:
    log_text -- string, text written to log file 
    '''
    
    with open(log_file, "a+") as f:
        log_text = "%s -%s" % (log_time, log_text_dict[event])
        f.write(log_text)
        return log_text

def write_data(browser, data, data_dir, data_file, log_text_dict, log_file, finish=False):
    '''
    Write scraped movie data to file.
    
    ARGUMENTS:
    browser -- current Selenium webdriver object
    data -- dictionary containing the following keys:
        "seat_config" -- dictionary with keys ["auditorium", "description", "seats", "seat_types"]
        "seat_reserv" -- dictionary with keys ["R", "U"], listing indices of reserved and unavailable seats
        "auditorium" -- string, auditorium name as specified by Fandango
    data_dir -- path to directory containing movie data file
    data_fle -- path to movie data file
    log_text_dict -- dictionary containing default log phrases for each possible event
    log_file -- path to movie data log file
    finish -- boolean, do or don't stop scraping data on this movie (default: False)
    
    RETURNS:
    write_log() -- writes event data to log file.
    '''
    
    log_time = str(datetime.now())
    
    seat_config = data["seat_config"]
    seat_reserv = data["seat_reserv"]
    auditorium = seat_config["auditorium"]
    
    img_dir = "../images/%s/%s" % (data_dir.split("/")[-1], auditorium)
    img_name = "%s.png" % auditorium
    img_path = "%s/%s" % (img_dir, img_name)
    
    if not os.path.exists(img_path):
        
        if not os.path.exists(img_dir):
            os.makedirs(img_dir)
        
        browser.set_window_size(1080, 1080)
        element = browser.find_element_by_xpath("//*[@id='seatpickerPage']")
        browser.execute_script("return arguments[0].scrollIntoView(true);", element)
        browser.save_screenshot(img_path)
    
    if not os.path.exists(data_file):
        
        if not os.path.exists(data_dir):
            os.makedirs(data_dir)
        
        with open(data_file, "w") as f:
            f.write(str(seat_config) + "\n")
            f.write(str([log_time, seat_reserv]) + "\n")
            return write_log("start", log_time, log_text_dict, log_file)

    else:
        
        with open(data_file, "a+") as f:
            
            f.seek(0)
            saved_seat_config = f.readline().strip("\n")
            
            if saved_seat_config.strip("\n") != str(seat_config):
                return write_log("error_seat_config", log_time, log_text_dict, log_file)
            
            else:
                f.write(str([log_time, seat_reserv]) + "\n")
                return write_log("finish" if finish else "append", log_time, log_text_dict, log_file)

In [ ]:
def get_seating_chart_and_write(movie_date, 
                                movie_time_full,
                                params,
                                movie_info,
                                url_base,
                                log_text_dict,
                                op_sys="win",
                                browser=None):
    '''
    Get movie seating chart reservation status and write data to file.
    
    ARGUMETS:
    movie_date -- string, date of movie as specified by Fandango
    movie_time_full -- string, time of movie and showing-type ID as specified by Fandango
    params -- URL parameters for a GET query (dictionary with the keys ["tid", "mid", "from"])
    movie_info -- dictionary with the keys ["dropdowndates", "showing_type_ids", 
        "movie_title", "theater_name", "error"]
    url_base -- URL to query
    log_text_dict -- dictionary containing default log phrases for each possible event
    op_sys -- "win" or "linux", operating system of machine running this code (default: "win") 
    browser -- current Selenium webdriver object
    
    RETURNS:
    skip -- boolean, do or don't skip data collection attempt for this movie showing
    browser -- current Selenium webdriver object
    '''
    
    skip = False
    
    [movie_time, showing_type_id] = movie_time_full.split("_")
    movie_datetime = datetime.strptime("%s %s" % (movie_date, movie_time), "%m/%d/%Y %I:%M %p")
    movie_datetime_formatted = movie_datetime.strftime("%Y-%m-%d_%H%M") + "_%s" % showing_type_id
    description = movie_info["showing_type_ids"][showing_type_id]

    delta = movie_datetime - datetime.now()
    finish = False
    if delta.total_seconds() < 60 * 8:
        finish = True

    url = params_to_url_string(params, url_base)
    
    browser, page_info = click_through(url, movie_date, movie_time_full, op_sys, browser)

    if page_info["error"]:
#         print("  |-- Movie Time [%s] %s" % (movie_datetime, page_info["error"]))
        skip = True
        return skip, browser

    seat_info = find_and_parse_seat_tags(page_info["source"])

    seats = seat_info["seats"]
    auditorium = seat_info["auditorium"]
    
    if seat_info["error"]:
        print("  |-- Movie Time [%s] %s" % (movie_datetime, seat_info["error"]))
        skip = True
        return skip, browser
    
    seat_config = {
        "auditorium": auditorium,
        "description": description,
        "seats": [x[0] for x in seats],
        "seat_types": [x[1] for x in seats]
    }
    
    data = {
        "seat_config": seat_config,
        "seat_reserv": {
            "R": [i for i in range(len(seats)) if seats[i][2] == "R"], 
            "U": [i for i in range(len(seats)) if seats[i][2] == "U"]
        }
    }

    paths = get_paths(params, movie_info, movie_datetime_formatted)

    log_text = write_data(browser,
                          data, 
                          paths["data_dir"], 
                          paths["data_file"], 
                          log_text_dict, 
                          paths["log_file"], 
                          finish)

#     print("  |-- Movie Time [%s] %s" % (str(movie_datetime), log_text))
    
    return skip, browser

# get_seating_chart_and_write("7/6/2018", "11:30 am_6819668", params, url_base, log_text_dict, op_sys)

In [ ]:
op_sys = "win"

url_base = "https://tickets.fandango.com/transaction/ticketing/express/ticketboxoffice.aspx"

incredibles_params = {
    "tid": "AABFB",                # theater id
    "mid": "185805",               # movie id
    "from": "mov_det_showtimes"    # page navigated from
}

antman_params_digital_3d = {
    "tid": "AABTB",
    "mid": "212354",
    "from": "mov_det_showtimes"
}

antman_params_imax_3d = {
    "tid": "AABTB",
    "mid": "211343",
    "from": "mov_det_showtimes"
}

antman_params_standard = {
    "tid": "AABTB",
    "mid": "203801",
    "from": "mov_det_showtimes"
}

antman_params_amc_standard = {
    "tid": "AAUNU",
    "mid": "203801",
    "from": "mov_det_showtimes"
}

antman_params_amc_digital_3d = {
    "tid": "AAUNU",
    "mid": "212354",
    "from": "mov_det_showtimes"
}

antman_params_big_newport_std = {
    "tid": "AABFB",
    "mid": "203801",
    "from": "mov_det_showtimes"
}

antman_params_big_newport_digital_3d = {
    "tid": "AABFB",
    "mid": "212354",
    "from": "mov_det_showtimes"
}

all_params = [
    antman_params_digital_3d,
    antman_params_imax_3d,
    antman_params_standard,
    antman_params_amc_standard,
    antman_params_amc_digital_3d,
    antman_params_big_newport_std,
    antman_params_big_newport_digital_3d
]

log_text_dict = {
    "start": " START: Successfully retrieved seating data.\n",
    "append": " Successfully appended seating data.\n",
    "finish": " FINISH: Successfully appended seating data.\n",
    "error_seat_config": "----- ERROR: Inconsistent seating configuration.\n"
}

###### --------------------------------------------------------------- #####

# def iterate_through_movie_times(url_base, params, log_text_dict, ops_sys, browser=None): 

#     days_before = 2
    
#     movie_info = get_movie_info(url_base, params)

#     if movie_info["error"]:
#         print(params["tid"], params["mid"], "\n", movie_info["error"])
#         return browser
    
#     movie_times = movie_info["dropdowndates"]

#     for date in (date for date in movie_times if (datetime.strptime(date, "%m/%d/%Y") - datetime.now()).days < days_before):

#         for item in movie_times[date]:

#             for t in item["Times"]:

#                 skip, browser = get_seating_chart_and_write(date, 
#                                                             t, 
#                                                             params, 
#                                                             movie_info, 
#                                                             url_base, 
#                                                             log_text_dict, 
#                                                             op_sys, 
#                                                             browser)

#                 if skip:
#                     continue
    
#     return browser

# def loop_until(end_datetime, url_base, params, log_text_dict, op_sys, browser=None):
    
#     loop_num = 0
    
#     while datetime.now() < end_datetime:
        
#         if loop_num != 0:
#             print("[%s]" % str(datetime.now()), params["tid"], params["mid"], " - Loop %d begun" % loop_num)
        
#         browser = iterate_through_movie_times(url_base, params, log_text_dict, op_sys, browser)
        
#         loop_num += 1
        
#     print("Looping ceased at %s." % str(datetime.now()))
    
#     return


# t = {}

# for i in range(len(all_params)):

#     params = all_params[i]
    
#     kwargs = {
#         "end_datetime": datetime(2018, 7, 7, 23, 59, 59),
#         "url_base": url_base,
#         "params": params,
#         "log_text_dict": log_text_dict,
#         "op_sys": op_sys,
#         "browser": None
#     }
    
#     t[i] = threading.Thread(target = loop_until, kwargs = kwargs)
#     t[i].start()
    
###### --------------------------------------------------------------- #####

def collect_data(loop_num, loop_dt=None):
    '''
    Loop once through all the movie times and showings for the specified
    movie parameters. NOTE: THIS IS A POORLY BEHAVING FUNCTION.
    
    ARGUMENTS:
    loop_num -- current loop number
    loop_dt -- length of time since the beginning of previous loop
    
    RETURNS:
    loop_num -- next loop number
    loop_dt -- length of time since the beginning of this loop
    '''
    
    days_before = 1
    
    browser = None
    loop_num += 1
    
    tic = datetime.now()
    
    try: 
        if loop_dt:
            print("[%s] - Loop %d begun. (Previous loop length: %.2f minutes)" % (str(tic), loop_num, loop_dt))
        else:
            print("[%s] - Loop %d begun." % (str(tic), loop_num))

        for params in all_params:

            movie_info = get_movie_info(url_base, params)

            if movie_info["error"]:
                print(params, "\n", movie_info["error"])
                venue_num += 1
                continue

            movie_times = movie_info["dropdowndates"]

            for date in (date for date in movie_times if (datetime.strptime(date, "%m/%d/%Y") - datetime.now()).days < days_before):

                for item in movie_times[date]:

                    for t in item["Times"]:

                        skip, browser = get_seating_chart_and_write(date, 
                                                                    t, 
                                                                    params, 
                                                                    movie_info, 
                                                                    url_base, 
                                                                    log_text_dict, 
                                                                    op_sys, 
                                                                    browser)

                        if skip:
                            continue

            browser.close()
            browser = None
        
        loop_dt = (datetime.now() - tic).total_seconds()/60
        
        return loop_num, loop_dt

    except:
        try:
            browser.close()
        except:
            pass
        print("\n>>>>>>>>>>>> Unknown error encountered. Re-starting process. <<<<<<<<<<<<\n")
        loop_dt = (datetime.now() - tic).total_seconds()/60
        
        return loop_num, loop_dt
    
    
def loop(start_hour = 7):
    '''
    Infinitely loop through all the movie times and showings for the specified
    movie parameters. Let the data collection process sleep between midnight and
    the specified start hour. NOTE: THIS IS A POORLY BEHAVING FUNCTION.
    
    ARGUMENT:
    start_hour -- int, time when the data collection process resumes after sleeping
        since midnight
    
    RETURNS: None
    '''
    
    loop_num = 0
    sleep_counter = 0
    loop_dt = None
    
    while True:
        
        if datetime.now().hour >= start_hour:
            loop_num, loop_dt = collect_data(loop_num, loop_dt)
            
        else:
            if sleep_counter == 0:
                print("[%s] sleeping until %d:00 . . ." % (str(datetime.now()), start_hour))
            sleep_counter += 1
            if sleep_counter > 6:
                sleep_counter = 0    
            sleep(5*60)
    
    return

# start_datetime = datetime(2018, 7, 8, 7, 0, 0)
# end_datetime = datetime(2018, 7, 8, 23, 59, 0)
# dt = (start_datetime - datetime.now()).total_seconds()

# print("Starting in %f seconds.\n" % dt)

# t = threading.Timer(dt, loop, kwargs = {"end_datetime": end_datetime})
# t.start()

loop(start_hour = 7)

[2018-07-16 20:39:08.911859] - Loop 1 begun.
[2018-07-16 20:45:56.579961] - Loop 2 begun. (Previous loop length: 6.79 minutes)
[2018-07-16 20:52:20.680672] - Loop 3 begun. (Previous loop length: 6.40 minutes)

>>>>>>>>>>>> Unknown error encountered. Re-starting process. <<<<<<<<<<<<

[2018-07-16 20:56:55.623865] - Loop 4 begun. (Previous loop length: 4.58 minutes)
[2018-07-16 21:03:00.997640] - Loop 5 begun. (Previous loop length: 6.09 minutes)
[2018-07-16 21:08:56.964011] - Loop 6 begun. (Previous loop length: 5.93 minutes)
[2018-07-16 21:15:02.471102] - Loop 7 begun. (Previous loop length: 6.09 minutes)
  |-- Movie Time [2018-07-17 13:50:00] We are temporarily unable to process your order for the theater selected. Fandango is currently working to correct this issue, please try again soon.

>>>>>>>>>>>> Unknown error encountered. Re-starting process. <<<<<<<<<<<<

[2018-07-16 21:21:27.933489] - Loop 8 begun. (Previous loop length: 6.42 minutes)
[2018-07-16 21:27:36.689816] - Loop 9 be

---
# Other
---

In [ ]:
# soup = BeautifulSoup(source, "html.parser")
# print(soup.decode(pretty_print = True))

In [ ]:
# tic = time()

# browser = msoup.StatefulBrowser()
# # my_url = "https://www.fandango.com/92612_movietimes"
# # my_url = "https://www.fandango.com/incredibles-2-185805/movie-times"
# # my_url = "https://tickets.fandango.com/transaction/ticketing/express/ticketboxoffice.aspx?row_count=231470826&tid=AABFB&sdate=2018-07-05+21:30&mid=185805&from=mov_det_showtimes"

# base_url = "https://tickets.fandango.com/transaction/ticketing/express/ticketboxoffice.aspx"

# showing_params = {
#     "row_count": "231470826",      # ???
#     "tid": "AABFB",                # theater id
#     "sdate": "2018-07-05+21:30",   # showing date
#     "mid": "185805",               # movie id
#     "from": "mov_det_showtimes"    # page navigated from
# }

# # response = browser.open(base_url, params = showing_params)
# # soup = BeautifulSoup(response.content, "html.parser")
# # dt = time() - tic
# # print("Time elapsed: %s s" % dt)
# response.content.decode("utf-8") 

In [ ]:
# showing_params = {
# #     "row_count": "231470826",      # ???
#     "tid": "AABFB",                # theater id
# #     "sdate": "2018-07-05+21:30",   # showing date
#     "mid": "185805",               # movie id
#     "from": "mov_det_showtimes"    # page navigated from
# }

# url_base = "https://tickets.fandango.com/transaction/ticketing/express/ticketboxoffice.aspx"

# response = requests.get(url = url_base, params = showing_params)
# response

In [ ]:
# response.url

In [ ]:
# content = response.content.decode("utf-8") 
# match = re.search(r"var dropdowndates = ({[\s\S]+]})\/\/\]\]", content)
# if match:
#     dropdowndates = json.loads(match.group(1))
# else:
#     print("No showing times found.")
    
# dropdowndates

In [ ]:
# showing_params = {
# #     "row_count": "231470826",      # ???
#     "tid": "AABFB",                # theater id
# #     "sdate": "7/5/2018+21:30",   # showing date
#     "mid": "185805",               # movie id
#     "from": "mov_det_showtimes"    # page navigated from
# }

# browser = msoup.StatefulBrowser()
# msoup_response = browser.open(response.url)
# browser.select_form()
# # browser.select_form().print_summary()
# browser["ShowDateDropDownList"] = "7/12/2018"
# browser.submit_selected()
# browser["ShowTimeDropDownList"] = "9:15 pm_6819668"
# # browser.get_current_form().print_summary()

In [ ]:
# soup = BeautifulSoup(msoup_response.content, "html.parser")
# print(soup.decode(pretty_print = True))

# Select 1 ticket to to view seat selection 
* `Adult: 1`
* `Senior: 0`
* `Child: 0`

(Assume that we already have a ticket selection URL, e.g. "https://tickets.fandango.com/transaction/ticketing/express/ticketboxoffice.aspx?row_count=231470826&tid=AABFB&sdate=2018-07-05+21:30&mid=185805&from=mov_det_showtimes")

In [ ]:
# ticket_selection_url = "https://tickets.fandango.com/transaction/ticketing/express/ticketboxoffice.aspx?row_count=232344705&tid=AABTB&sdate=2018-07-12+21:15&mid=185805&from=mov_det_showtimes"

In [ ]:
# browser = msoup.StatefulBrowser()
# msoup_response = browser.open(ticket_selection_url)
# form = browser.select_form()
# browser["AreaRepeater$ctl00$TicketRepeater$ctl00$quantityddl"] = "1"
# # soup = BeautifulSoup(t_response.content, "html.parser")
# # form.choose_submit("inputTotal")
# response_1 = browser.submit_selected()

In [ ]:
# soup.button

In [ ]:
# soup = BeautifulSoup(response_1.content, "html.parser")
# print(soup.decode(pretty_print = True))